Refereces:
- https://www.google.com/search?q=how+to+use+tkinter+to+create+a+chatbot+in+python&oq=how+to+use+tkinter+to+create+a+chatbot&aqs=chrome.1.69i57j33.44216j0j4&sourceid=chrome&ie=UTF-8#kpvalbx=_VJZOX5znAquIytMPkfS2iAE37

In [1]:
# install packages/libraries first 
# !pip install pyttsx3
# !pip install wikipedia
# !pip install SpeechRecognition
# !pip install pygame
# !pip install pyown

In [2]:
# !pip install pyaudio
# !pip install pyaudio

In [3]:
# from tkinter import *
# root=Tk()

In [4]:

import pandas as pd
import numpy as np
import tensorflow as tf
import time
from nltk.corpus import stopwords
import re, string
import nltk
import random

### import libraries for voice output
# source: https://www.codementor.io/@edwardzionsaji/simple-voice-enabled-chat-bot-in-python-kt2qi5oke
import datetime
import webbrowser
import pyttsx3
import wikipedia
from pygame import mixer
import speech_recognition as sr

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
# Set up and calibrate the text to speech engine.
# Now we need to set the voice rate, engine, etc.
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)
volume = engine.getProperty('volume')
engine.setProperty('volume', 10.0)
rate = engine.getProperty('rate')
engine.setProperty('rate', rate - 25)

In [6]:
# load the datasets
df=pd.read_csv("chatbot_data_Q&A - basic_python_questions.csv").dropna(axis=0)

In [7]:
df.isnull().sum()# check the missing values

question    0
answer      0
dtype: int64

In [8]:
df.head()# take a look at the first five rows

,question,answer
0,data?,Based on the definition from google : facts an...
1,what is data science?,Data science is an inter-disciplinary field th...
2,data science,Data science is an inter-disciplinary field th...
3,what is big data?,"In the data science domain, big data usually r..."
4,definition of big data,"In the data science domain, big data usually r..."


In [9]:
df.shape # check the shape

(118, 2)

In [10]:
# Remove some uncompleted questions
# if a question or answers end with ":" 
questions=[]
for i in df.question:
    if i[-1]==":":
        questions.append(None)
    else:
        questions.append(i)
        
        
# remove some uncompleted answers
answers=[]
for i in df.answer:
    if i[-1]==":":
        answers.append(None)
    else:
        answers.append(i)

In [11]:
# rearrange a little bit
df['answer']=answers
df['question']=questions

In [12]:
df=df.dropna(axis=0).copy()
df.head()

,question,answer
0,data?,Based on the definition from google : facts an...
1,what is data science?,Data science is an inter-disciplinary field th...
2,data science,Data science is an inter-disciplinary field th...
3,what is big data?,"In the data science domain, big data usually r..."
4,definition of big data,"In the data science domain, big data usually r..."


In [13]:
df.shape

(116, 2)

In [14]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r'[-()\"#/@;:<>{}`+=~|.!?,]\|', "", text)
    text=text.replace("[\'","").replace("\n"," ").replace("']"," ").replace('["',"").replace('"]',"").replace("it\'s","it's ").replace("\', \'","")
    text=text.replace("\',","").replace("it\'s","it is ").replace("it\\\'s","it is").replace(" \\"," ")
    text=text.replace('",'," ").replace("\',","").replace( ":\',","").replace("here\'s","").replace(":","").replace(',"',"")
    text=text.replace("[","").replace("]","").replace("\'s","'s")
    text=re.sub(r"let's", "let us", text)
    text = text.replace("\'s", "")
    return text

In [15]:
#declare answers and questions
questions=df.question
answers=df.answer
# Cleaning the questions
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
# Cleaning the answers
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [16]:
clean_questions[-5:]

['definition of data visualization, what is data visualization?',
 'what tools do professional data scientists use? ',
 'mian tools in data science',
 'how to install git',
 'what is conda and anaconda?']

In [17]:
# list contains  punctuation
#sw_list = stopwords.words('english')
sw_list = list(string.punctuation)
sw_list += ["''", '""', '...', '``', '’', '“', '’', '”', '‘', '‘',"'", '©',
'said',"'s", "also",'one',"n't",'com', '-', '–','--' ,
'—', '_']
sw_set = set(sw_list)

# tokenization
def process_data(string):
    tokens = nltk.word_tokenize(string) # tokenization
    punctuation_removed = [token.lower() for token in tokens if token.lower() not in sw_set]
    return punctuation_removed

# Stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()
# create a function stemming() and loop through each word in a review
def stemming(string):
    stemmed_string=[]
    for w in string:
        stemmed_string.append(ps.stem(w))
    return stemmed_string

# import libraries
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# create a function  and loop through each word in  a review
def lemmatization(string):
    lemma_list=[]
    for word in string:
        lemma_word=lemmatizer.lemmatize(word,pos='v') 
        lemma_list.append(lemma_word)
    return lemma_list

# Conbime all functions above and obtian cleaned text data 
def data_preprocessing(text_data):
    #tokenization, stop words removal, punctuation marks removel
    processed_string=list(map(process_data,text_data))
    # stemming
    stemming_string=list(map(stemming,processed_string))
    # lemmatization
    lemma_string=list(map(lemmatization,stemming_string))
    
    return lemma_string

In [18]:
# using the function above to process the data 
cleaned_questions=data_preprocessing(clean_questions)

In [19]:
print(cleaned_questions[:10])

[['data'], ['what', 'be', 'data', 'scienc'], ['data', 'scienc'], ['what', 'be', 'big', 'data'], ['definit', 'of', 'big', 'data'], ['big', 'data'], ['famou', 'data', 'scientist'], ['what', 'be', 'matplotlib'], ['what', 'be', 'seaborn'], ['what', 'be', 'panda']]


In [20]:
# create a function of NLP for single line
def NLP(text):
    cleaned_question=clean_text(text)
    processed_question=process_data(cleaned_question)
    stemming_question=stemming(processed_question)
    lemma_question=lemmatization(stemming_question)
    return lemma_question

In [21]:
# List of Greeting , goodbye and thank you you are welcome
greeting=['hey', 'hi', 'hello', 'hey man', 'hi how are you', 'how are you','how is it going', 
          'nice to meet you', 'how are you doing', 'what is up', 'what is new', 
          'what is going on', 'how is everything', 'how are things', 'how is life', 
          'how is your day', 'how is your day going', 'good to see you', 'nice to see you']
goodbye=["see you","bye","byebye","goodbye"]

thankyou=["thanks","thank you", "thank you very much"]
yourwelcome=["you are welcome ^.^","my pleasure!","I am happy to help you!"]

In [22]:
#=============================================================================================================#
'''
Please run part 1 and part 2 individually.
If you run part 1 first and run part 2 please " comment" part 1 then part 2
'''
#=============================================================================================================#

'\nPlease run part 1 and part 2 individually.\nIf you run part 1 first and run part 2 please " comment" part 1 then part 2\n'

# Part One - With Voice

In [26]:
# function - chatbot
def chatbot(question):
   
    if  question.strip().lower() in greeting:
        answer=random.choice(greeting).capitalize()
        return  answer
        
        
    elif question.strip().lower() in thankyou:
        answer= random.choice(yourwelcome).capitalize()
        return  answer
    else:
        # NLP
        pro_text= NLP(question)

        # to find which row has intersection with the words from question you asked
        # which means to find the who have common elements between your question and the data
        inter_list=[]
        for i in cleaned_questions:
            if (set(pro_text) & set(i)):
                inter_list.append((list(set(pro_text) & set(i)),cleaned_questions.index(i)))

        # remove stop words  
        new_inter_list=[]
        for i in range(len(inter_list)):
            for j in inter_list[i][0]:
                if j not in stopwords.words('english'):
                    new_inter_list.append(inter_list[i])

        # find the max length of common elements
        lengths=[len(new_inter_list[i][0]) for i in range(len(new_inter_list)) ]
        

        indexes=[]
        if len(lengths)>0:
            max_length=max(lengths)
            # find all the index whose correspondiong question data have the most common elements
            for i in range(len(new_inter_list)):
                if len(new_inter_list[i][0])==max_length:
                    indexes.append(new_inter_list[i][1])
        else:
            return "Sorry, I don't know. I need to learn more!"



        ratios=[]
        for i in list(set(indexes)):
            ratio=len(pro_text)/len(questions.iloc[i])
            ratios.append((ratio,i))

        


        final_indexes=[]
        if [ratios[i][0] for i in range(len(ratios))]!=0:
            max_ratios=max([ratios[i][0] for i in range(len(ratios))])
            for i in range(len(ratios)):
                if ratios[i][0]==max_ratios:
                    final_indexes.append(ratios[i][1])


        if len(final_indexes)>0:
            # to randomly find an answer based on the index
            answer_index=random.choice(final_indexes)
            answer=answers.iloc[answer_index]    
            return answer

        else:
            return "Sorry, I don't know. I need to learn more!"


In [30]:

print("=============# Hi There my name is Chacha. Let's Talk #=============")

while(True): 
    question = input("You: ")
    if question.strip().lower() in goodbye:
        answer=random.choice(goodbye)
        print("Chacha: "+answer+"\n") 
        engine.say(answer)
        engine.runAndWait()
        break
    else:
        pro_text= NLP(question)
        answer=chatbot(question)
        print("Chacha: "+answer+"\n")
        engine.say(answer)
        engine.runAndWait()

=============# Hi There my name is Chacha. Let's Talk #=============
You: hi
Chacha: What is up

You: what is data science?
Chacha: Data science is an inter-disciplinary field that uses scientific methods, processes, algorithms ... in managing a digital data collection. There is still no consensus on the definition of data science and it is considered by some to be a buzzword.

You: how to select a column from a dataframe?
Chacha: # if you want to select a specific column you can use this code:
df["column_name"] 

You: how to install git
Chacha: Git insallation on Windows: https://www.youtube.com/watch?v=2j7fD92g-gE
Git insallation on Mac: https://www.youtube.com/watch?v=sJ4zr0a4GAs

You: what is normal distribution?
Chacha: The normal distribution is the most important probability distribution in statistics because it fits many natural phenomena. For example, heights, blood pressure, measurement error, and IQ scores follow the normal distribution. It is also known as the Gaussian dist

# Part Two - chatbot with interface

In [ ]:
# def func(event):
#      print("You hit return.")
# def send(event=None):
#     send="You: "+e.get()
#     txt.insert(END,"\n"+send)
#     txt.see(END+"\n")
    
#     if e.get().strip().lower() in goodbye:
#         byebye="Byebye! see you next time! "
#         txt.insert(END,"\n"+"Chacha: "+byebye+"\n")  
#         e.delete(0,END)
#         txt.see(END)

#     elif  e.get().strip().lower() in greeting:
#         answer=random.choice(greeting).capitalize()
#         txt.insert(END,"\n"+"Chacha: "+ answer+"\n")
#         e.delete(0,END)
#         txt.see(END)


#     elif e.get().strip().lower() in thankyou:
#         txt.insert(END,"\n"+"Chacha: "+ random.choice(yourwelcome).capitalize()+"\n")
#         e.delete(0,END)
#         txt.see(END)
#     else:
#         # NLP
#         pro_text= NLP(e.get())
#         # to find which row has intersection with the words from question you asked
#         # which means to find the who have common elements between your question and the data
#         inter_list=[]
#         for i in cleaned_questions:
#             if (set(pro_text) & set(i)):
#                 inter_list.append((list(set(pro_text) & set(i)),cleaned_questions.index(i)))         

#         # remove stop words  
#         new_inter_list=[]
#         for i in range(len(inter_list)):
#             for j in inter_list[i][0]:
#                 if j not in stopwords.words('english'):
#                     new_inter_list.append(inter_list[i])

#         # find the max length of common elements
#         lengths=[len(new_inter_list[i][0]) for i in range(len(new_inter_list)) ]
#         indexes=[]
#         if len(lengths)>0:
#             max_length=max(lengths)
#             # find all the index whose correspondiong question data have the most common elements
#             for i in range(len(new_inter_list)):
#                 if len(new_inter_list[i][0])==max_length:
#                     indexes.append(new_inter_list[i][1])
#                     # to find ratios that the keys in a sentence
#             ratios=[]
#             for i in list(set(indexes)):
#                 ratio=len(pro_text)/len(questions.iloc[i])
#                 ratios.append((ratio,i))
#             final_indexes=[]
#             if [ratios[i][0] for i in range(len(ratios))]!=[]:
#                 max_ratios=max([ratios[i][0] for i in range(len(ratios))])

#                 for i in range(len(ratios)):
#                     if ratios[i][0]==max_ratios:
#                         final_indexes.append(ratios[i][1])
#             else:
#                 txt.insert(END,"\n"+"Chacha: "+"Sorry, I don't know. I need to learn more!"+"\n")
#                 e.delete(0,END)
#                 txt.see(END)



#             if len(final_indexes)>0:
#                 # to randomly find an answer based on the index
#                 answer_index=random.choice(final_indexes)
#                 txt.insert(END,"\n"+ "Chacha: "+ answers.iloc[answer_index]+"\n")
#                 e.delete(0,END)
#                 txt.see(END)
#             else:
#                 txt.insert(END,"\n"+"Chacha: "+"Sorry, I don't know. I need to learn more!"+"\n")
#                 e.delete(0,END)
#                 txt.see(END)        
                    
#         else:
#             txt.insert(END,"\n"+"Chacha: "+"Sorry, I don't know. I need to learn more!"+"\n")
#             e.delete(0,END)
#             txt.see(END)

 
#     e.delete(0,END)
    
    
# root.bind('<Return>', send)    
# txt=Text(root)
# txt.grid(row=0,column=0,columnspan=2)
# e=Entry(root,width=60)
# send=Button(root,text="Send",command=send,fg='black', bg='white').grid(row=1,column=1)
# e.grid(row=1,column=0)
# root.title("Mini Chatbot for Data Science")
# # root.configure(bg='white')
# root.mainloop()